### Setup

In [11]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

IPython autoawait is `on`, and set to use `asyncio`


In [12]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

In [13]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)

Subscribing too: present_proof


### Go to seller notebook to build connection

In [14]:
try:
    response = await agent_controller.connections.get_connections()
    results = response['results']
    print("Results : ", results)
    print('\n')
    if len(results) > 0:
        connection = response['results'][0]
        print("Connection :", connection)
        if connection['state'] == 'active':
            connection_id = connection["connection_id"]
            print("\nActive Connection ID : ", connection_id)
        else:
            print("\nNo active connection found - wait a bit and execute again")
    else:
        print("You must create a connection")
except ConnectionRefusedError as e:
    print(repr(e))

Results :  [{'connection_id': 'e812df45-5cab-4e9b-a099-cd801de4fd33', 'accept': 'manual', 'invitation_key': 'EzJK5B9Whm1rm4utkVJj8HpevUg6ppU3ete5YfQLRKTn', 'invitation_mode': 'once', 'routing_state': 'none', 'their_did': 'WGBGMeCT19cC5Thdjey9fa', 'created_at': '2023-01-03 00:10:27.158633Z', 'state': 'active', 'their_role': 'inviter', 'updated_at': '2023-01-03 00:10:47.369114Z', 'my_did': '5mjER6QMs8dEZdfJvfNcXZ', 'their_label': 'Bob', 'rfc23_state': 'completed', 'request_id': '84a406b6-85c3-49c1-afca-681a76aa2a3d'}]


Connection : {'connection_id': 'e812df45-5cab-4e9b-a099-cd801de4fd33', 'accept': 'manual', 'invitation_key': 'EzJK5B9Whm1rm4utkVJj8HpevUg6ppU3ete5YfQLRKTn', 'invitation_mode': 'once', 'routing_state': 'none', 'their_did': 'WGBGMeCT19cC5Thdjey9fa', 'created_at': '2023-01-03 00:10:27.158633Z', 'state': 'active', 'their_role': 'inviter', 'updated_at': '2023-01-03 00:10:47.369114Z', 'my_did': '5mjER6QMs8dEZdfJvfNcXZ', 'their_label': 'Bob', 'rfc23_state': 'completed', 'request

In [15]:
import time

response = await agent_controller.wallet.get_public_did()
print(response)
issuer_did = response["result"]["did"]

print("Request proof of the device")
#Set some variables

revocation = False
exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "ProductionDate", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "Manufacturer", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "Owner", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "Type", "restrictions": [{"issuer_did": issuer_did}]}
]


curtime = int(time.time())
#Set predicates for Zero Knowledge Proofs
req_preds = [
    # test zero-knowledge proofs
    {
        "name": "ExpirationDate",
        "p_type": ">=",
        "p_value": curtime,
        "restrictions": [{"issuer_did": issuer_did}],
    },
    {
        "name": "Status",
        "p_type": ">",
        "p_value": 0,
        "restrictions": [{"issuer_did": issuer_did}],
    }
]

indy_proof_request = {
    "name": "Proof of Personal Information",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}
indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

{'result': {'did': 'PQRXDxdGqQGSZ8z69p4xZP', 'verkey': 'DDEKJtBjzaXAJtpwetdPiXH5s4rNUEzG18V15QoLRcZZ', 'posture': 'public'}}
Request proof of the device


In [16]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

Handle present proof
Role verifier, Exchange f5d621a4-1eb8-45cd-93ba-9cc6a5237d1e in state request_sent
{'connection_id': 'e812df45-5cab-4e9b-a099-cd801de4fd33', 'trace': False, 'presentation_exchange_id': 'f5d621a4-1eb8-45cd-93ba-9cc6a5237d1e', 'initiator': 'self', 'created_at': '2023-01-03 00:35:48.552485Z', 'state': 'request_sent', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '499e67da-067c-4b2a-a9dd-bb79bd4c7306', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIFBlcnNvbmFsIEluZm9ybWF0aW9uIiwgInZlcnNpb24iOiAiMS4wIiwgInJlcXVlc3RlZF9hdHRyaWJ1dGVzIjogeyIwX1Byb2R1Y3Rpb25EYXRlX3V1aWQiOiB7Im5hbWUiOiAiUHJvZHVjdGlvbkRhdGUiLCAicmVzdHJpY3Rpb25zIjogW3siaXNzdWVyX2RpZCI6ICJQUVJYRHhkR3FRR1NaOHo2OXA0eFpQIn1dfSwgIjBfTWFudWZhY3R1cmVyX3V1aWQiOiB7Im5hbWUiOiAiTWFudWZhY3R1cmVyIiwgInJlc3RyaWN0aW9ucyI6IFt7Imlzc3Vlcl9kaWQiOiA

### Go to seller notebook to send verify response

In [17]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify)

ClientResponseError: 400, message='Presentation exchange f5d621a4-1eb8-45cd-93ba-9cc6a5237d1e in request_sent state (must be presentation_received)', url=URL('http://alice-agent:8021/present-proof/records/f5d621a4-1eb8-45cd-93ba-9cc6a5237d1e/verify-presentation')

Error during POST /present-proof/records/f5d621a4-1eb8-45cd-93ba-9cc6a5237d1e/verify-presentation: 400, message='Presentation exchange f5d621a4-1eb8-45cd-93ba-9cc6a5237d1e in request_sent state (must be presentation_received)', url=URL('http://alice-agent:8021/present-proof/records/f5d621a4-1eb8-45cd-93ba-9cc6a5237d1e/verify-presentation')


### If credential revoked, verify['verified'] should become Fales

In [8]:
print(verify['state'])
print(verify['state'] == 'verified')
print(verify['verified'])

verified
True


In [9]:
for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print(name + " : " + val['raw'])

0_Type_uuid : Dentistry
0_ProductionDate_uuid : 1672706006
0_Manufacturer_uuid : GOOD_COMPANY
0_Owner_uuid : GOOD_SELLER


In [10]:
response = await agent_controller.terminate()